In [3]:
import torch

In [ ]:
checkpoints = torch.load('../output/model_Epoch_00030_Iter_0000029.pth', 'cpu')['model']
new_keys = list(list(checkpoints.keys()))
new_keys[:10]

In [1]:
import os
os.getcwd()

'/mnt/lustre/zhujinguo/jinguo_data/codes/Uni-Perceiver/work_dirs'

In [4]:
ds_checkpoints = torch.load('deepspeed_moe/BERT_L12_H192_experiments/7task_150k_berttiny_lr1e-3_wd0.05_gc0.1_prenorm_warm10k_layerscale1e-3_uniformdp0.1_maeinit_fixedpos_torchfp16_unifieddataset_224inputsize_tagmoe_alllayer/tagmoe_alllayer_exp4/149999/mp_rank_00_model_states.pt', 'cpu')['module']
# ds_checkpoints = torch.load('/nfs/zhujinguo/codes/xmodaler/work_dirs/deepspeed_moe/BERT_L12_H768_experiments/basetagmoe_pretrainstage2/89999/mp_rank_00_model_states.pt', 'cpu')['module']
# ds_checkpoints = torch.load('/nfs/zhujinguo/codes/xmodaler/work_dirs/deepspeed_moe/BERT_L12_H768_experiments/bertbase_womoe/89999/mp_rank_00_model_states.pt', 'cpu')['module']




In [6]:
oldkeys = list(ds_checkpoints.keys())
# oldkeys[:20]

In [5]:
mapping_dict = {

    'encoder.': 'fused_encoder.',
    # 'attention.self.qkv_proj.weight': 'self_attn.in_proj_weight',
    # 'attention.self.qkv_proj.bias': 'self_attn.in_proj_bias',
    'attention.self.qkv_proj': 'self_attn.qkv_proj',
    'deepspeed_moe.gate': 'gate',
    'deepspeed_moe.experts': 'experts',
    'attention.output.dense': 'self_attn.dense',
    'attention_output.residual_scale': 'gamma_1',
    'ffn.dense.': 'linear1.',
    'ffn.dense2.': 'linear2.',
    'ffn_output.residual_scale': 'gamma_2',
    'LayerNormModules.0.': 'norm1.',
    'LayerNormModules.1.': 'norm2.',
    'predictor.': 'loss_prepare.',
    
}

In [7]:
new_checkpoint = {}
for k, v in ds_checkpoints.items():
    if k.endswith('residual_scale'):
        v = v.squeeze(1).squeeze(0)
        # print(v.shape )
        
    if k.startswith('visual_embed'):
        continue
    
        
    for origin_str, target_str in mapping_dict.items():
        if origin_str in k:
            k = k.replace(origin_str, target_str)
    # merge type embedding in video_embed 
    # if k=='video_embed.embeddings.bias':
    #     v = v + ds_checkpoints['video_embed.embeddings_type.weight'][0]

    new_checkpoint[k] = v.float()
    # if 'wg' in k:
    #     print(f'{k}, {v}')
# new_checkpoint['video_embed.embeddings.bias'] = new_checkpoint['video_embed.embeddings.bias'] + new_checkpoint['video_embed.embeddings_type.weight'][0]
    

In [8]:
torch.save({ 'model': new_checkpoint}, 'pretrained_models/uni-perceiver-moe-tiny-L12-H192-224size-pretrained-withvtype.pth')


# torch.save({ 'model': new_checkpoint}, 'pretrained_models/uni-perceiver-moe-tiny-L12-H192-224size-pretrained.pth')
# torch.save({ 'model': new_checkpoint}, 'pretrained_models/uni-perceiver-moe-base-L12-H768-224size-pretrained.pth')
# torch.save({ 'model': new_checkpoint}, 'pretrained_models/uni-perceiver-base-L12-H768-224size-pretrained-custom-attn-module.pth')


